In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')


from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/opt/workspace/{}/notebooks/Clickstream_Analytics/scr'.format(curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser)) 
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))


# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from spark_connector import SparkConnector
from sparkdb_loader import spark
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

# import loader as load

# from corpora_process import utils
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''
    
    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)
    
    return _typed_udf_wrapper

In [6]:
%%time
sp = spark(schema='sklod_external_google_analytics',
           queue = 'g_dl_u_corp.ektov1-av_ca-sbrf-ru',#'root.g_dl_u_dateng.ektov1-av_ca-sbrf-ru', #'g_dl_u_corp.ektov1-av_ca-sbrf-ru',
           numofinstances=12, 
           numofcores=8,
           dynamic_alloc=False,
           kerberos_auth=False) 
print(sp.sc.version)
hive = sp.sql

2.2.0.cloudera4
CPU times: user 59.2 ms, sys: 19.4 ms, total: 78.6 ms
Wall time: 40.6 s


In [ ]:
# @typed_udf(IntegerType())
# def checkhitPageTitleList(x, l):
#     values = []
#     if x is not None:
#         for pattern in l:
#             res = re.findall(x.lower(), r'.*{}.*'.format(pattern.lower()))
#             if len(res)!=0:
#                 values.append(1)
#             else:
#                 values.append(0)
#     else:
#         values.append(0)
#     return values        

In [7]:
#@typed_udf(StringType())

def checkhitPageTitle(dctofrefwords, title, path):
#     pattern2= re.compile(r'([a-zA-Z=\-\_\&,.#\/0-9]+)(?:\?.*)*')
    finout = None
    for key, wordlst in dctofrefwords.items():
        for reword in wordlst:
            wpatt = re.compile(reword)
            if title is not None:
                res = wpatt.findall(title.lower())          
                if len(res)!=0:
                    if path is not None:
    #                     out=pattern2.findall(path)
                        finout = key
                        break
                    else:
                        None
                else:
                     None
            else:
                None
                
        if finout is not None: 
            break
                
    return finout
    
def udf_checkhitPageTitle (lst):
    return f.udf(lambda t,p: checkhitPageTitle(lst,t,p))

## Set the focus on the visit datamart

In [ ]:
gadf = hive.sql("select * from dl_external_google_analytics.visit")

In [ ]:
gadf.filter('lower(eventLabel) like "%scrol%"').show(15)

In [ ]:
gadf.filter(f.col('deviceIsMobile')==True).count()

# Scenario - product page activity

In [7]:
src_path = os.getcwd() + '/src/'
sys.path.insert(0, src_path) 

from urlprocessing import ParseURL
from collections import OrderedDict
from datetime import datetime

In [8]:
sp.sc.addPyFile(src_path+'/urlprocessing.py')
GetUrl = sp.sc.broadcast(ParseURL())

In [9]:
get_url_domain_udf = f.udf(GetUrl.value.get_url_domain, StringType())
get_url_path_udf   = f.udf(GetUrl.value.get_url_path, StringType())

### Test cases

In [ ]:
gadf = gadf.withColumn("url_domain", get_url_domain_udf("hitPagePath")) \
           .withColumn("url_path", get_url_path_udf("hitPagePath"))

In [ ]:
gadf.filter("hitPagePath like '%/s_m_business/%'").select('hitPagePath', 'url_domain', 'url_path').show(30)

# Main selection with conditions

In [7]:
refdate = '2020-05-01'

In [8]:
sql = \
'''
    select 
    cid, 
    sessionId, 
    visitNumber,
    hitType, 
    hitPagePath, 
    hitPageTitle, 
    sessionStartTime, 
    hitNumber, 
    hitTime, 
    eventCategory, 
    eventAction, 
    eventLabel,
    deviceIsMobile,
    deviceMobileDeviceModel,
    deviceDeviceCategory,
    geoRegion,
    geoCity
from sklod_external_google_analytics.visit
where 
    hitPageHostName like '%www.sberbank.ru%'
    and hitPagePath like '%/s_m_business/%' 
    and hitPageHostName not in ('sbi.sberbank.ru', 'localhost') 
    --and hitType = 'PAGE' 
    and timestamp(sessionDate) >= timestamp('{}') 
'''.format(refdate)

In [ ]:
gadf_trimmed = hive.sql(sql)

In [11]:
gadf_trimmed = gadf_trimmed.withColumn('sessionTime', 
                                       f.from_unixtime(f.col('sessionStartTime')+f.round(f.col('hitTime')/1000))
                                      )
gadf_trimmed = gadf_trimmed.orderBy(f.col('sessionTime').asc())

In [12]:
gadf_trimmed.cache()

DataFrame[cid: string, sessionId: bigint, visitNumber: bigint, hitType: string, hitPagePath: string, hitPageTitle: string, sessionStartTime: bigint, hitNumber: bigint, hitTime: bigint, eventCategory: string, eventAction: string, eventLabel: string, deviceIsMobile: boolean, deviceMobileDeviceModel: string, deviceDeviceCategory: string, geoRegion: string, geoCity: string, sessionTime: string]

In [13]:
gadf_trimmed.count()

122681588

In [15]:
gadf_trimmed.select("cid").distinct().count()

7095416

In [124]:
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    f.lower(f.col("eventLabel")).like("%right%")                   
                   ).select("cid").distinct().count()

0

In [ ]:
# 1st banner: Кешбек до 10%
gadf_trimmed.filter((
                     f.col("eventAction").like("%click_any%") & 
                     f.lower(f.col("eventLabel")).like("%подключить%")
                    )|
                    (f.col("eventAction").like("%click_banner%") & 
                     f.lower(f.col("eventLabel")).like("%кешбэк%")
                    )                     
                   ).show()

In [105]:
# 2nd banner: Новые пакеты услуг
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click_any%") & 
                     f.lower(f.col("eventLabel")).like("%подключить%"))|
                    (f.col("eventAction").like("%click_banner%") & 
                     f.lower(f.col("eventLabel")).like("%пакеты услуг для ведения%"))                     
                   ).select("cid").distinct().count()

18

In [104]:
# 3rd banner: Мобильное приложение Сберсоветник
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click_any%") & 
                     f.lower(f.col("eventLabel")).like("%подробнее%"))|
                    (f.col("eventAction").like("%click_banner%") & 
                     f.lower(f.col("eventLabel")).like("%аналитика для развития%"))                     
                   ).select("cid").distinct().count()

0

In [103]:
# 5th banner: Мгновенные переводы 
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click_any%") & 
                     f.lower(f.col("eventLabel")).like("%подробнее%"))|
                    (f.col("eventAction").like("%click_banner%") & 
                     f.lower(f.col("eventLabel")).like("%онлайн-переводы%"))                     
                   ).select("cid").distinct().count()

20

In [119]:
# 6th banner: Откройте бизнесс от 0Р
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click_any%") & 
                     f.lower(f.col("eventLabel")).like("%узнать больше%"))|
                    (f.col("eventAction").like("%click_banner%") & 
                     f.lower(f.col("eventLabel")).like("%откройте бизнес%"))                     
                   ).select("cid").distinct().count()

5230

In [101]:
# 8th banner: Прием платежей
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click_any%") & 
                     f.lower(f.col("eventLabel")).like("%узнать больше%"))|
                    (f.col("eventAction").like("%click_banner%") & 
                     f.lower(f.col("eventLabel")).like("%прием платежей%"))                     
                   ).select("cid").distinct().count()

0

In [111]:
# Placement #1: Новые пакеты услуг РКО
gadf_trimmed.filter((~(f.col("hitPagePath").like("%/ru/s_m_business/test%")))&
                    f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click%") & 
                     f.lower(f.col("eventLabel")).like("%новые пакеты услуг%"))                     
                   ).select("cid").distinct().count()

1

In [112]:
# Placement #2:  Опции к патекам услуг
gadf_trimmed.filter((~(f.col("hitPagePath").like("%/ru/s_m_business/test%")))&
                    f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click%") & 
                     f.lower(f.col("eventLabel")).like("%опции к пакетам услуг%"))                     
                   ).select("cid").distinct().count()

0

In [117]:
# Placement #6:  Опции к патекам услуг
gadf_trimmed.filter((~(f.col("hitPagePath").like("%/ru/s_m_business/test%")))&
                    f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click%") & 
                     f.lower(f.col("eventLabel")).like("%как принимать%"))                     
                   ).select("cid").distinct().count()

0

In [118]:
gadf_trimmed.filter(f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%") &
                    (f.col("eventAction").like("%click%") & 
                     f.lower(f.col("eventLabel")).like("%undefined_content-card%"))                     
                   ).show()

,cid,sessionId,visitNumber,hitType,hitPagePath,hitPageTitle,sessionStartTime,hitNumber,hitTime,eventCategory,eventAction,eventLabel,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity,sessionTime,show_banner,click_banner,prev_state_m1,prev_state_m2,show_banner_time,diff_clk_show,click_banner_state,check_stay_on_banner,click_banner_time


In [23]:
gadf_trimmed.select(f.from_unixtime(f.min("sessionStartTime")).alias("min_sessionStartTime"),
                    f.from_unixtime(f.max("sessionStartTime")).alias("max_sessionStartTime")).show()

,min_sessionStartTime,max_sessionStartTime
0,2020-05-01 00:00:00,2020-09-08 23:59:59


In [17]:
w=Window.partitionBy('cid').orderBy('sessionId')
gadf_trimmed = gadf_trimmed.withColumn('show_banner', (f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%"))&
                                                      (f.col('eventAction').like("%show_banner%")))\
                           .withColumn('click_banner', (f.col("eventCategory").like("%SITE_Corporate_.ru.s_m_business%"))&
                                                       (f.col('eventAction').like("%click_banner%")))\
                           .withColumn('click_banner_state', f.max(f.when(f.col('eventAction').like("%click_banner%"), 
                                                              True).otherwise(False)).over(w))

In [25]:
gadf_trimmed.filter("show_banner == True").count()

1254

In [120]:
(45079 / 5545189)*100

0.8129389277804597

In [18]:
res = \
gadf_trimmed.groupBy("cid").agg(f.sum(f.col("show_banner").cast(IntegerType())).alias("sum_show_banner"),
                                f.sum(f.col("click_banner").cast(IntegerType())).alias("sum_click_banner"))

res.select(f.sum("sum_show_banner").alias("sum_total_show_banner"),
           f.sum("sum_click_banner").alias("sum_total_click_banner")).show()

,sum_total_show_banner,sum_total_click_banner
0,1254,160


In [45]:
gadf_trimmed.filter((f.col("cid") == "1016857375.1596799214")&
                    (f.col("sessionTime") > f.to_timestamp(f.lit("2020-08-18 14:00:00")))).show(70)

,cid,sessionId,visitNumber,hitType,hitPagePath,hitPageTitle,sessionStartTime,hitNumber,hitTime,eventCategory,eventAction,eventLabel,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity,sessionTime,show_banner,click_banner,prev_state_m1,prev_state_m2,show_banner_time,diff_clk_show,click_banner_state,check_stay_on_banner
0,1016857375.1596799214,1597748393,3,PAGE,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,17,319272,None,None,None,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:12,None,True,None,None,2020-08-18 14:05:13,0,1,0
1,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,18,319952,SITE_Widget_CorpBannerCarousel,show_banner_0,text_Кредитная поддержка малого бизнеса,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:13,True,True,None,None,2020-08-18 14:05:13,0,1,0
2,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,19,337355,SITE_Corporate_credits.restr,click_any,Программа кредитования на возобновление деятельности под 2% годовых_t-h4,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:30,False,True,show_banner_0,None,2020-08-18 14:05:13,0,1,0
3,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,20,341948,SITE_Corporate_credits.restr,click_any,Выбрать решение_kit-button__text,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_any,show_banner_0,2020-08-18 14:05:13,0,1,0
4,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,21,342101,SITE_Widget_CorpBannerCarousel,click_banner,Кредитная поддержка малого бизнеса Если ваш бизнес столкнулся с трудно,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_any,click_any,2020-08-18 14:05:13,22,1,0
5,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,22,342103,SITE_Widget_CorpBannerCarousel,click_banner_0,Кредитная поддержка малого бизнеса Если ваш бизнес столкнулся с трудно,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_banner,click_any,2020-08-18 14:05:13,22,1,0
6,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,23,349105,SITE_Corporate_credits.restr,informing_leads,30 sec on page,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:42,False,True,click_banner_0,click_banner,2020-08-18 14:05:13,0,1,1
7,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,24,435814,SITE_Corporate_credits.restr,click_any,Вопросы по программе ›_,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:07:09,False,True,informing_leads,click_banner_0,2020-08-18 14:05:13,0,1,0
8,1016857375.1596799214,1597748393,3,PAGE,/ru/s_m_business/credits/restr?tab=2,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,25,436644,None,None,None,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:07:10,None,True,click_any,informing_leads,2020-08-18 14:05:13,0,1,0
9,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr?tab=2,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,27,451814,SITE_Corporate_credits.restr,click_go_to_partner,https:..economy.gov.ru.material.news.ekonomika_bez_virusa.minekonomrazvitiya_sformirovalo_reestr_sonko_dlya_okazaniya_mer_podderzhki_v_period_rasprostraneniya_koronavirusa.html,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:07:25,False,True,None,click_any,2020-08-18 14:05:13,0,1,0


In [62]:
w=Window.partitionBy('cid').orderBy('sessionId')
gadf_trimmed = gadf_trimmed.withColumn('prev_state_m1', f.lag(f.col('eventAction'),count=1).over(w))
gadf_trimmed = gadf_trimmed.withColumn('prev_state_m2', f.lag(f.col('eventAction'),count=2).over(w))
gadf_trimmed = gadf_trimmed.withColumn('show_banner_time', f.min(f.when(f.col('eventAction').like("%show_banner%"), 
                                                                        f.col("sessionTime")).otherwise(None)).over(w))
gadf_trimmed = gadf_trimmed.withColumn('click_banner_time', f.min(f.when(f.col('eventAction').like("%click_banner%"), 
                                                                        f.col("sessionTime")).otherwise(None)).over(w))
gadf_trimmed = gadf_trimmed.withColumn('diff_clk_show', f.max(f.when(f.col('click_banner')==True,
                                                               f.abs(f.unix_timestamp('show_banner_time') -
                                                                     f.unix_timestamp('click_banner_time')).cast(IntegerType())).otherwise(0)
                                                             ).over(w))

In [65]:
gadf_trimmed.filter((f.col("cid") == "1016857375.1596799214")&
                    (f.col("sessionTime") > f.to_timestamp(f.lit("2020-08-18 14:00:00")))).show(70)

,cid,sessionId,visitNumber,hitType,hitPagePath,hitPageTitle,sessionStartTime,hitNumber,hitTime,eventCategory,eventAction,eventLabel,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity,sessionTime,show_banner,click_banner,prev_state_m1,prev_state_m2,show_banner_time,diff_clk_show,click_banner_state,check_stay_on_banner,click_banner_time
0,1016857375.1596799214,1597748393,3,PAGE,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,17,319272,None,None,None,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:12,None,None,None,None,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
1,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,18,319952,SITE_Widget_CorpBannerCarousel,show_banner_0,text_Кредитная поддержка малого бизнеса,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:13,True,False,None,None,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
2,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,19,337355,SITE_Corporate_credits.restr,click_any,Программа кредитования на возобновление деятельности под 2% годовых_t-h4,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:30,False,False,show_banner_0,None,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
3,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,20,341948,SITE_Corporate_credits.restr,click_any,Выбрать решение_kit-button__text,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,False,click_any,show_banner_0,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
4,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,21,342101,SITE_Widget_CorpBannerCarousel,click_banner,Кредитная поддержка малого бизнеса Если ваш бизнес столкнулся с трудно,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_any,click_any,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
5,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,22,342103,SITE_Widget_CorpBannerCarousel,click_banner_0,Кредитная поддержка малого бизнеса Если ваш бизнес столкнулся с трудно,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_banner,click_any,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
6,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,23,349105,SITE_Corporate_credits.restr,informing_leads,30 sec on page,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:42,False,False,click_banner_0,click_banner,2020-08-18 14:05:13,22,True,1,2020-08-18 14:05:35
7,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,24,435814,SITE_Corporate_credits.restr,click_any,Вопросы по программе ›_,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:07:09,False,False,informing_leads,click_banner_0,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
8,1016857375.1596799214,1597748393,3,PAGE,/ru/s_m_business/credits/restr?tab=2,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,25,436644,None,None,None,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:07:10,None,None,click_any,informing_leads,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
9,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr?tab=2,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,27,451814,SITE_Corporate_credits.restr,click_go_to_partner,https:..economy.gov.ru.material.news.ekonomika_bez_virusa.minekonomrazvitiya_sformirovalo_reestr_sonko_dlya_okazaniya_mer_podderzhki_v_period_rasprostraneniya_koronav

In [64]:
gadf_trimmed = gadf_trimmed.withColumn('check_stay_on_banner', f.when( (f.col('eventAction').like('%informing_leads%'))&
                                                                       (f.col('eventLabel').like("%scroll 30 percent%")|
                                                                        f.col('eventLabel').like("%scroll 50 percent%")|
                                                                        f.col('eventLabel').like("%scroll 75 percent%")|
                                                                        f.col('eventLabel').like("%sec on page%")
                                                                       )&
                                                                       (f.col("click_banner_state")==1)&
                                                                       (f.col("diff_clk_show")<=30)
                                                                       , 1
                                                                       ).otherwise(0))

In [67]:
gadf_trimmed.filter("check_stay_on_banner == 1").select("cid").distinct().count()

45079

In [130]:
gadf_trimmed.filter("cid = 1016857375.1596799214 and click_banner == True").show(50)

,cid,sessionId,visitNumber,hitType,hitPagePath,hitPageTitle,sessionStartTime,hitNumber,hitTime,eventCategory,eventAction,eventLabel,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity,sessionTime,show_banner,click_banner,prev_state_m1,prev_state_m2,show_banner_time,diff_clk_show,click_banner_state,check_stay_on_banner,click_banner_time
0,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,21,342101,SITE_Widget_CorpBannerCarousel,click_banner,Кредитная поддержка малого бизнеса Если ваш бизнес столкнулся с трудно,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_any,click_any,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35
1,1016857375.1596799214,1597748393,3,EVENT,/ru/s_m_business/credits/restr,«Сбербанк» - Кредитная поддержка малого бизнеса,1597748393,22,342103,SITE_Widget_CorpBannerCarousel,click_banner_0,Кредитная поддержка малого бизнеса Если ваш бизнес столкнулся с трудно,False,None,desktop,Smolensk Oblast,(not set),2020-08-18 14:05:35,False,True,click_banner,click_any,2020-08-18 14:05:13,22,True,0,2020-08-18 14:05:35


In [25]:
# res = gadf_trimmed.filter(f.col('eventCategory').like('%chto-takoe-ekvajring-i-kak-on-rabotaet%'))
# res = res.orderBy(f.col('sessionTime').asc())

In [ ]:
res.show(50)

In [29]:
gadf_trimmed = gadf_trimmed.withColumn("url_domain", get_url_domain_udf("hitPagePath")) \
                           .withColumn("url_path", get_url_path_udf("hitPagePath"))

In [30]:
gadf_trimmed.cache()

DataFrame[cid: string, sessionId: bigint, visitNumber: bigint, hitType: string, hitPagePath: string, hitPageTitle: string, sessionStartTime: bigint, hitNumber: bigint, hitTime: bigint, eventCategory: string, eventAction: string, eventLabel: string, deviceIsMobile: boolean, deviceMobileDeviceModel: string, deviceDeviceCategory: string, geoRegion: string, geoCity: string, url_domain: string, url_path: string]

In [31]:
gadf_trimmed.count()

38937451

In [34]:
gadf_trimmed.select('cid','hitPagePath', 'url_domain', 'url_path').show(50)

,cid,hitPagePath,url_domain,url_path
0,1845923051.1540309456,/ru/s_m_business/help/newsbbol,ru,s_m_business/help/newsbbol
1,527514055.1581927943,/ru/s_m_business/open-accounts,ru,s_m_business/open-accounts
2,728915766.1590200976,/ru/s_m_business/onlinecredit,ru,s_m_business/onlinecredit
3,375942385.1591037733,/ru/s_m_business/help/aquiring/oplata,ru,s_m_business/help/aquiring/oplata
4,158274897.1591023340,/ru/s_m_business/credits/restr?tab=2,ru,s_m_business/credits/restr
5,61159944.1591004813,/ru/s_m_business/credits,ru,s_m_business/credits
6,1255002434.1590581811,/ru/s_m_business/new_sbbol,ru,s_m_business/new_sbbol
7,269424543.1578643358,/ru/s_m_business/reserved-checking-account24?channel=mkkavdyushina.a.se,ru,s_m_business/reserved-checking-account24
8,269424543.1578643358,/ru/s_m_business/reserved-checking-account24?channel=mkkavdyushina.a.se,ru,s_m_business/reserved-checking-account24
9,906603655.1590988113,/ru/s_m_business/pro_business/chto-takoe-elektronnaya-podpis-kak-poluchit-i-dlya-chego-nuzhna/,ru,s_m_business/pro_business/chto-takoe-elektronnaya-podpis-kak-poluchit-i-dlya-chego-nuzhna


In [ ]:
gadf_trimmed.filter('cid = 1845923051.1540309456')\
            .select('sessionId', 
                    'visitNumber',
                    'hitType',
                    'hitPagePath',
                    'eventCategory', 
                    'eventAction', 
                    'eventLabel',
                    f.from_unixtime('sessionStartTime').alias('sessionStartTime'), 
                    'hitTime').orderBy(f.col('visitNumber').asc(), 
                                       f.col('hitTime').asc()).show(90)

### Parse `hitPageTitle` in accordance with the specified target sentences

In [19]:
dctofrefwords = {'рко'      :
                             ['.*рко.*', 
                              '.*расчетный сч[её]т.*', 
                              '.*расч[её]тно-кассовое обслуживание.*', 
                              '.*сч[её]т\s*[для]*[\s\w]*[ип|ооо|бизнеса]*.*', 
                              '.*сч[её]т\s*[для]*юридического лица.*', 
                              '.*открыть сч[её]т\s*[для]*[\s\w]*[ип|ооо|юридического лица]*.*', 
                              '.*открытие сч[её]та онлайн.*'],
                 'кредит'   :
                             ['.*кредит\s*[для]*[\s\w]*[бизнеса|малого бизнеса]*.*',  
                              '.*инвестиционный кредит.*', 
                              '.*кредитная линия.*', 
                              '.*кредитование бизнеса.*',
                              '.*оборотный кредит.*', 
                              '.*кредит на пополнение оборотных средств.*', 
                              '.*кредит под залог.*', 
                              '.*кредит на[\s\w]*[автомобиль|оборудование|товар|недвижимость|транспорт|бизнес цели|развитие бизнеса]*.*', 
                             ],
                 'бизнес-карта': 
                             ['.*бизнес[\-\s]*карта\s*[для]*.*', 
                              '.*карта для бизнеса.*', 
                              '.*корпоративная карта.*', 
                              '.*карта для[\s\w]*[ип|ооо]*.*'
                             ],
                 'эквайринг': 
                             [
                              '.*эквайринг.*', 
                              '.*торговый\s*эквайринг.*', 
                              '.*прием оплаты картой.*'
                             ],
                 'зарплатный проект':
                             [
                              '.*зарплатный проект.*',
                              '.*[оформление|обслуживание]* зарплатной карты.*',  
                              '.*зарплатная карта.*'  
                             ],
                 'депозит': 
                           [
                             '.*депозит.*', 
                             '[размещение|привлечение]* денежных средств.*', 
                             '.*вклад.*' 
                           ],
                 'электронный документооборот':
                           [
                             '.*электронный документооборот.*',
                             '.*e-invoicing.*', 
                             '.*эдо.*', 
                             '.*сэд.*', 
                             '.*цифровой документооборот.*'
                           ],
                 'электронный архив':
                           [
                             '.*электронный архив.*', 
                             '.*оцифровка архивов.*', 
                             '.*цифровые документы.*'
                           ],
                 'электронная подпись':
                           [
                             '.*[электронная|цифровая][\s\w]*подпись.*',
                             '.*э[цп]?.*',
                             '.*кэп.*',  
                             '.*подпись для[\s\w]*[тендеров|торгов|госуслуг|налоговой|егаис]*.*'
                           ],
                 'овердрафт':
                           [
                            '.*овердрафт.*', '.*овердрафтный кредит.*'
                           ],
                 'интернет эквайринг':
                           [
                            '.*интернет[\s\w\-]*эквайринг.*'    
                           ],
                 'лизинг': 
                           ['.*лизинг.*'],
                 'инкассация': 
                           ['.*инкассация.*'],
                 'инкассация на ус': 
                           [
                            '.*самоинкассация.*', 
                            '.*инкассация на ус.*', 
                            '.*внесение средств.*', 
                            '.*внесение выручки.*'    
                           ],                
                 'центр управления счетами':
                           [
                             '.*управление счетами.*'  
                           ],
                'аккредитивы': ['.*аккредитив.*'],
                'e-invoicing': ['.*электронная отчетность.*', 
                                '.*конструктор документов', 
                                '.*электронный архив.*'],
                'яндекс маркет': ['.*яндекс[\s\w\-]*маркет.*'],
                'конверсионные операции': ['.*конверсионные операции.*', 
                                           '.*обмен валюты.*', 
                                           '.*валютно-обменные операции.*', 
                                           '.*валютные [свопы|рынки]*.*'],
                'таможенные платежи': ['.*таможенные платежи', 
                                       '.*таможенная карта.*'],
                'факторинг': ['.*факторинг.*', 
                              '.*беззалоговое финансирование.*'],
                'консалтинг по ВЭД': ['.*[консалтинг|консультации]*[\s\w]*по вэд.*', 
                                      '.*консалтинговые услуги по вэд.*'],
                'эвотор': ['.*эвотор.*'],
                'эскроу': ['.*эскроу.*'],
                'кредитная бизнес-карта': ['.*кредитная бизнес[\s\w\-]*карта.*'],
                'спец счет для торгов': ['.*счет для [торгов|закупок]*.*', 
                                         '.*счет для участия в [закупках|тендерах]*.*', 
                                         '.*спцсчет для [торгов|госконтракта]*.*', 
                                         '.*счет[\s\w]*[по\s44\-фз]*.*', 
                                         '.*счет участника закупок.*', 
                                         '.*счет[\s\w]*[по\s223\-фз]*.*'],
                'валютный контроль': ['вэд', 
                                      '.*внешнеэкономическая деятельность.*', 
                                      '.*валютный контроль.*'],
                'DocDoc': ['.*docdoc.*', 
                           '.*докдок.*', 
                           '.*телемедицина.*'],
                'plazius': ['.*plazius.*'],
                'защита сотрудников': ['.*защита сотрудников.*', 
                                       '.*страхование сотрудников.*'],
                'международная платформа ВВР': ['.*bbp.*', 
                                                '.*bank of business partners.*', 
                                                '.*платформа для экспортеров и импортеров.*', 
                                                '.*экспортный акселератор.*'],
                'банковское сопровождение поектов': ['.*банковское сопровождение.*'],
                'гособоронзаказ': ['.*гособоронзаказ.*', 
                                   '.*счет[\s\w]*[по\s\гоз]*.*'],
                'прием платежей физ. лиц': ['.*прием платежей от[\s\w]*[физлиц|физических\sлиц|физ\sлиц]*.*'],
                'онлайн кассы': ['.*онлайн[\s\-]*касса.*',
                                 '.*касса[\s\w]*[по\s54\-фз]*.*'],
                'продвижение бизнеса онлайн': ['.*продвижение бизнеса онлайн.*', 
                                               '.*сервис для продвижения бизнеса.*', 
                                               '.*продвижение бизнеса в интернете.*'],
                'сайт для бизнеса': ['.*конструктор сайта.*', 
                                     '.*сайт для бизнеса.*']
                }

In [20]:
gadf_trimmed = gadf_trimmed.withColumn('prod_gr', udf_checkhitPageTitle(dctofrefwords)('hitPageTitle','url_path'))

In [ ]:
gadf_trimmed.filter('url_path_filtered is not Null').select('hitPageTitle', 'hitPagePath', 'url_path', 'prod_gr').show(10)

In [ ]:
gadf_trimmed.cache()

# Analyse User Session in Details

### `hitTimefromStartTime`: calculate total hitTime starting from `sessionStartTime`

In [21]:
res = gadf_trimmed.withColumn('hitTimefromStartTime', 
                              f.from_unixtime(f.col('sessionStartTime')+f.col('hitTime')/1000, 
                                                      format='yyyy-MM-dd HH:mm:ss').cast(TimestampType()))

In [ ]:
res.filter('prod_gr ="рко" and hitType = "PAGE"').show(30)

## Assign a unique ID to each `sessionId` in accordance with the value of `hitType` column: 
- 1.Propagate the same ID between two non-succsessive cases with `hitType==PAGE` separated by the case with `hitType==EVENT`
- 2.Each new found case with `hitType==PAGE` is considered as a starting point to repeat p.1 
- 3.Each two/three succsessive cases with `hitType==PAGE` not separated by the case with `hitType==EVENT` shoud be retreated in terms of reassigning its own ID on one unique ID taken from the last cases with `hitType==PAGE` 

In [22]:
w1=Window.orderBy('cid','prod_gr','url_path','row_num')
w2=Window.orderBy('idx') 

res = res.orderBy(f.col('hitTimefromStartTime').asc())\
         .withColumn('idx', f.monotonically_increasing_id())\
         .withColumn('row_num', f.row_number().over(w2))
res = res.drop('idx')

#### One can use native `visitNumber` column that is relevant to our `agg_session_t` custom column

In [23]:
# skip this stage if we trust to visitNumber column
# substitute all following references to agg_session_t on visitNumber

res = res\
         .withColumn('page_select', f.when(f.col('hitType')=='PAGE',f.col('row_num')).otherwise(None))\
         .withColumn('agg_session_t', f.coalesce(f.col('page_select'),f.last(f.col('page_select'),True).over(w1)))
res = res.drop('page_select')        

In [24]:
res=res.withColumn('agg_session', f.when(((f.lead(f.col('hitType'),count=1).over(w1)=='PAGE')&\
                                          (f.lead(f.col('hitType'),count=2).over(w1)=='PAGE'))&(f.col('hitType')!='EVENT'),
                                                 f.lead(f.col('agg_session_t'),count=2).over(w1))\
                                   .when(((f.lead(f.col('hitType'),count=1).over(w1)=='PAGE')&\
                                          (f.lead(f.col('hitType'),count=2).over(w1)!='PAGE'))&(f.col('hitType')!='EVENT'),
                                                 f.lead(f.col('agg_session_t'),count=1).over(w1))\
                                   .otherwise(f.col('agg_session_t'))) 
res = res.drop('agg_session_t')

In [ ]:
res.filter('cid = 888169552.1558288660')\
            .select('sessionId', 
                    'prod_gr',
                    'url_path',
                    'visitNumber',
                    'agg_session',
                    'hitType',
                    'hitPagePath',
                    f.from_unixtime('sessionStartTime').alias('sessionStartTime'), 
                    'hitTime').orderBy(f.col('visitNumber').asc(), 
                                       f.col('sessionStartTime').asc()).show(50)

#### Use native `visitNumber`

In [ ]:
res = res.orderBy(f.col('hitTimefromStartTime').asc())
res=res.withColumn('agg_session', f.when(((f.lead(f.col('hitType'),count=1).over(w1)=='PAGE')&\
                                          (f.lead(f.col('hitType'),count=2).over(w1)=='PAGE'))&(f.col('hitType')!='EVENT'),
                                                 f.lead(f.col('visitNumber'),count=2).over(w1))\
                                   .when(((f.lead(f.col('hitType'),count=1).over(w1)=='PAGE')&\
                                          (f.lead(f.col('hitType'),count=2).over(w1)!='PAGE'))&(f.col('hitType')!='EVENT'),
                                                 f.lead(f.col('visitNumber'),count=1).over(w1))\
                                   .otherwise(f.col('visitNumber'))) 

res = res.drop('row_num')

In [ ]:
res.filter('prod_gr ="рко" and cid = 55986308.1566479327').show(50)

## Dumping intermediate results into tempTable

In [ ]:
res.registerTempTable('tmp_cls_scenarios')
# hive.cacheTable('tmp_cls_scenarios')
hive.sql("drop table if exists T_TEAM_DS_KB_SME.tmp_GA_seg_scenarios")

hive.sql('create table T_TEAM_DS_KB_SME.tmp_GA_seg_scenarios select * from tmp_cls_scenarios')

## Build additional aggregates based on the Window with `agg_session` 

In [ ]:
res = hive.sql("select * from T_TEAM_DS_KB_SME.tmp_GA_seg_scenarios")

In [ ]:
res.count()

In [ ]:
res.filter('prod_gr is not Null').select('url_path', 'prod_gr').distinct()\
                            .groupby(['prod_gr']).agg(f.size(f.collect_set('url_path')).alias('prod_gr_cnt')).show(30)

### `firstPageHit`: Calculate first hit time on control Page

In [ ]:
res = res.withColumn('firstPageHit', f.min(f.when(f.col('hitType')=='PAGE', 
                                                  f.col('hitTimefromStartTime')).otherwise(None)
                                          )\
                                      .over(Window.partitionBy('cid','prod_gr','url_path','agg_session')))
res = res.withColumn('firstPageHitUrl', f.first(f.when((f.col('hitType')=='PAGE')&\
                                                       (f.col('hitTimefromStartTime') == f.col('firstPageHit')), 
                                                       f.col('url_path')).otherwise(None)
                                               ).over(Window.partitionBy('cid','prod_gr','url_path','agg_session')))

### Aggregate different `sessionId` for the same `cid` into groups by criteria: if `hitTimefromStartTime` still less than `firstPageHit` +30 min) then assign ID==1

In [ ]:
dttm_sess_right1 = f.from_unixtime(f.unix_timestamp(f.col('firstPageHit')) + 1*30*60, 
                                               'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
dttm_sess_left1  = f.from_unixtime(f.unix_timestamp(f.col('firstPageHit')) + 0*30*60, 
                                               'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

dttm_sess_right2 = f.from_unixtime(f.unix_timestamp(f.col('firstPageHit')) + 2*30*60, 
                                               'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
dttm_sess_left2  = f.from_unixtime(f.unix_timestamp(f.col('firstPageHit')) + 1*30*60, 
                                               'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
    
dttm_sess_right3 = f.from_unixtime(f.unix_timestamp(f.col('firstPageHit')) + 3*30*60, 
                                               'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
dttm_sess_left3  = f.from_unixtime(f.unix_timestamp(f.col('firstPageHit')) + 2*30*60, 
                                               'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

In [ ]:
res = res.withColumn('trg_sess_between' , f.when((f.col('hitTimefromStartTime')>=dttm_sess_left1) & \
                                                 (f.col('hitTimefromStartTime')<dttm_sess_right1), 1)\
                                           .when((f.col('hitTimefromStartTime')>=dttm_sess_left2) & \
                                                 (f.col('hitTimefromStartTime')<dttm_sess_right2), 2)\
                                           .when((f.col('hitTimefromStartTime')>=dttm_sess_left3) & \
                                                 (f.col('hitTimefromStartTime')<dttm_sess_right3), 3).otherwise(0))

In [ ]:
res = res.withColumn('lastPageHit',  f.max(f.when((f.col('hitType')=='PAGE'), 
                                                  f.col('hitTimefromStartTime')
                                                 ).otherwise(None)
                                          )\
                                      .over(Window.partitionBy('cid','prod_gr','url_path',
                                                               'agg_session','trg_sess_between')))
res = res.withColumn('firstEventHit', f.min(f.when(f.col('hitType')=='EVENT', 
                                                   f.col('hitTimefromStartTime')).otherwise(None)
                                           )\
                                      .over(Window.partitionBy('cid','prod_gr','url_path',
                                                               'agg_session','trg_sess_between')))
res = res.withColumn('lastEventHit',  f.max(f.when((f.col('hitType')=='EVENT') , 
                                                   f.col('hitTimefromStartTime')).otherwise(None)
                                           ).over(Window.partitionBy('cid','prod_gr','url_path',
                                                                     'agg_session','trg_sess_between')))

In [ ]:
res = res.withColumn('countPageHit',  f.sum(f.when(f.col('hitType')=='PAGE', 1).otherwise(0)
                                           )\
                                       .over(Window.partitionBy('cid', 'prod_gr', 'url_path', 
                                                                'agg_session','trg_sess_between')))
res = res.withColumn('countEventHit', f.sum(f.when(f.col('hitType')=='EVENT', 1).otherwise(0)
                                           )\
                                       .over(Window.partitionBy('cid', 'prod_gr', 'url_path', 
                                                                'agg_session','trg_sess_between')))

### CHECKING BLOCK: Filtering 

In [ ]:
res.filter('prod_gr ="рко" and cid = 55986308.1566479327').show(60)

# Explore records with `eventAction == click_any` 

## Build analytics on selected `eventLabel` values 

In [ ]:
reEventsDct=\
{'click_any':
[
    ('оформить', '.*оформить.*'),
    ('открыть', '.*открыть.*'),
    ('тарифы', '.*тарифы.*'),
    ('выпустить' ,'.*выпустить.*'),
    ('оставить заявку', '.*оставить заявку.*'),
    ('отправить заявку', '.*отправить заявку.*'),
    ('узнать больше', '.*узнать больше.*'),
    ('подать заявку', '.*подать заявку.*'),
    ('я клиент сбербанка', '.*я клиент сбербанка.*'),
    ('я неклиент сбербанка', '.*я неклиент сбербанка.*'),
    ('выбрать', '.*выбрать.*'),
    ('купить', '.*купить.*'),
    ('отправить', '.*отправить.*'),
    ('выставить счет', '.*выставить счет.*'),
    ('подробнее', '.*подробнее.*'),
    ('рассчитать', '.*рассчитать.*'),
    ('подобрать', '.*подобрать.*'),
    ('взять в аренду', '.*взять в аренду.*'),
    ('открытие', '.*открытие.*'),
    ('подключить', '.*подключить.*'),
    ('войти', ".*войти.*"), 
    ('зарегистрироваться', ".*зарегистрироваться.*"), 
    ('подать заявку онлайн', ".*подать заявку онлайн.*"), 
    ('получить предложение', ".*получить предложение.*"), 
    ('клиент сбербанка', ".*клиент сбербанка.*"), 
    ('неклиент сбербанка', ".*неклиент сбербанка.*"), 
    ('получить номер счета', ".*получить номер сч[её]та.*"), 
    ('получить промо-код', ".*получить промо-код.*"), 
    ('подобрать тариф', ".*подобрать тариф.*")
]
}

In [ ]:
for col, repattern in reEventsDct['click_any']:
    res = res.withColumn('{}'.format(col), f.regexp_extract(f.lower(f.col('eventLabel')),'({})'.format(repattern), 1)) 

## Upload to Hive a Filtered Table

In [ ]:
res = res.filter('countPageHit > 0 and countEventHit > 0 and prod_gr is not Null')

In [ ]:
hive.sql('drop table if exists T_TEAM_DS_KB_SME.GA_seg_scenarios')

In [ ]:
res.registerTempTable('tmp_cls_scenarios')
# hive.cacheTable('tmp_cls_scenarios')

hive.sql('create table T_TEAM_DS_KB_SME.GA_seg_scenarios select * from tmp_cls_scenarios')

In [ ]:
hive.sql('drop table if exists T_TEAM_DS_KB_SME.tmp_GA_seg_scenarios')

## Filter some active users that have a higher inclination to buy a specific Sber products

In [ ]:
res = hive.sql("select * from T_TEAM_DS_KB_SME.GA_seg_scenarios")

In [ ]:
res = res.filter('prod_gr is not Null').select('url_path', 'prod_gr').distinct()\
                            .groupby(['prod_gr']).agg(f.sort_array(f.collect_set('url_path'), asc=False).alias('url_paths_set'))
prod_urls_df = res.toPandas()

In [ ]:
dct = prod_urls_df.to_dict(orient='records')

In [ ]:
res.filter('lower(eventLabel) like "%scrol%"').show(15)

### Select records within specified period 

In [ ]:
part_spdf = res.filter("(hitTimefromStartTime >= timestamp('2019-10-01 00:00:00')) and (hitTimefromStartTime < timestamp('2019-10-30 00:00:00'))")

In [ ]:
part_spdf.select('cid').distinct().count()

In [ ]:
part_spdf.select(f.min('countEventHit'), f.max('countEventHit')).show()

In [ ]:
part_spdf.filter('countEventHit > 200').show(30)

### Make filtering with `countEventHit` > 0 and exclude help pages 

In [ ]:
part_spdf\
.filter((f.col('eventAction')=='click_any')&(f.col('получить номер счета')!='')\
                                           &(f.col('countEventHit') > 0)\
                                           &(f.regexp_extract(f.lower(f.col('url_path')),'(s_m_business/help)', 1)!='s_m_business/help')\
                                           &(f.regexp_extract(f.lower(f.col('url_path')),'(img/uploaded)', 1)!='img/uploaded')
        ).show(30)

In [ ]:
part_spdf\
.filter((f.col('eventAction')=='click_any')&( (f.col('оформить')!='')\
                                             |(f.col('открыть')!='')\
                                             |(f.col('тарифы')!='')\
                                             |(f.col('выпустить')!='')\
                                             |(f.col('оставить заявку')!='')\
                                             |(f.col('отправить заявку')!='')\
                                             |(f.col('узнать больше')!='')\
                                             |(f.col('подать заявку')!='')\
                                             |(f.col('я клиент сбербанка')!='')\
                                             |(f.col('я неклиент сбербанка')!='')\
                                             |(f.col('выбрать')!='')\
                                             |(f.col('купить')!='')\
                                             |(f.col('отправить')!='')\
                                             |(f.col('выставить счет')!='')\
                                             |(f.col('подробнее')!='')\
                                             |(f.col('рассчитать')!='')\
                                             |(f.col('подобрать')!='')\
                                             |(f.col('взять в аренду')!='')\
                                             |(f.col('открытие')!='')\
                                             |(f.col('подключить')!='')\
                                             |(f.col('войти')!='')\
                                             |(f.col('зарегистрироваться')!='')\
                                             |(f.col('подать заявку онлайн')!='')\
                                             |(f.col('получить предложение')!='')\
                                             |(f.col('клиент сбербанка')!='')\
                                             |(f.col('неклиент сбербанка')!='')\
                                             |(f.col('получить номер счета')!='')\
                                             |(f.col('получить промо-код')!='')\
                                             |(f.col('подобрать тариф')!='')
                                            )\
                                           &(f.col('countEventHit') > 0)\
                                           &(f.regexp_extract(f.lower(f.col('url_path')),'(s_m_business/help)', 1)!='s_m_business/help')\
                                           &(f.regexp_extract(f.lower(f.col('url_path')),'(img/uploaded)', 1)!='img/uploaded')
        ).show(30)

In [ ]:
part_spdf\
.filter((f.col('eventAction')=='click_any')&( (f.col('оформить')!='')\
                                             |(f.col('открыть')!='')\
                                             |(f.col('тарифы')!='')\
                                             |(f.col('выпустить')!='')\
                                             |(f.col('оставить заявку')!='')\
                                             |(f.col('отправить заявку')!='')\
                                             |(f.col('узнать больше')!='')\
                                             |(f.col('подать заявку')!='')\
                                             |(f.col('я клиент сбербанка')!='')\
                                             |(f.col('я неклиент сбербанка')!='')\
                                             |(f.col('выбрать')!='')\
                                             |(f.col('купить')!='')\
                                             |(f.col('отправить')!='')\
                                             |(f.col('выставить счет')!='')\
                                             |(f.col('подробнее')!='')\
                                             |(f.col('рассчитать')!='')\
                                             |(f.col('подобрать')!='')\
                                             |(f.col('взять в аренду')!='')\
                                             |(f.col('открытие')!='')\
                                             |(f.col('подключить')!='')\
                                             |(f.col('войти')!='')\
                                             |(f.col('зарегистрироваться')!='')\
                                             |(f.col('подать заявку онлайн')!='')\
                                             |(f.col('получить предложение')!='')\
                                             |(f.col('клиент сбербанка')!='')\
                                             |(f.col('неклиент сбербанка')!='')\
                                             |(f.col('получить номер счета')!='')\
                                             |(f.col('получить промо-код')!='')\
                                             |(f.col('подобрать тариф')!='')
                                            )\
                                           &(f.col('countEventHit') > 0)\
                                           &(f.regexp_extract(f.lower(f.col('url_path')),'(s_m_business/help)', 1)!='s_m_business/help')\
                                           &(f.regexp_extract(f.lower(f.col('url_path')),'(img/uploaded)', 1)!='img/uploaded')
        ).select('cid').distinct().count()

### Aplly additional filtering rules

In [ ]:
out = \
res.filter((f.col('prod_gr') == "бизнес-карта")&\
           (f.col('eventAction') !='')&\
           (f.regexp_extract(f.lower(f.col('url_path')),'(help)', 1)!='help')&\
           (f.regexp_extract(f.lower(f.col('eventLabel')),'^подробнее о акциях_(?=(\b|$))', 0) == 'подробнее о акциях_'))

In [ ]:
out.count()

In [ ]:
out.select('cid').distinct().count()

In [ ]:
out = \
res.filter((f.col('prod_gr') == "бизнес-карта")&\
           (f.col('eventAction') !='')&\
           (f.regexp_extract(f.lower(f.col('url_path')),'(help)', 1)!='help')&\
           (f.regexp_extract(f.lower(f.col('eventLabel')),'^оформить бизнес-карту_(?=(\b|$))', 0) == 'оформить бизнес-карту_'))

In [ ]:
out.select('cid').distinct().count()

In [ ]:
out = \
res.filter((f.col('prod_gr') == "бизнес-карта")&\
           (f.col('eventAction') !='')&\
           (f.regexp_extract(f.lower(f.col('url_path')),'(help)', 1)!='help')&\
           (f.regexp_extract(f.lower(f.col('eventLabel')),'^оформить бизнес-карту_b-btn', 0) == 'оформить бизнес-карту_b-btn'))

In [ ]:
out.select('cid').distinct().count()

### Analyse mobile events

In [ ]:
out = \
res.filter((f.col('prod_gr') == "бизнес-карта")&\
           (f.col('eventAction') !='')&\
           (f.col('deviceIsMobile')==True)&\
           (f.regexp_extract(f.lower(f.col('url_path')),'(help)', 1)!='help'))\
   .groupBy('eventLabel').agg({'eventLabel':'count'}).orderBy(f.col('count(eventLabel)').desc())

In [ ]:
out.show(50)

In [ ]:
out.select('eventLabel',
           f.regexp_extract(f.lower(f.col('eventLabel')),'mobile', 0).alias('regexp')).filter("regexp!=''").show(20)

## Auxilliary Test

In [ ]:
df = sp.sc.parallelize([('a',None),('a',1),('a',-1),('a',None),('a',2),('a',-4),('b',3),('b',1),('b',1)]).toDF(['k','v'])

In [ ]:
w=Window.partitionBy('k').orderBy('v').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
w1=Window.partitionBy('k').orderBy('v')
w2=Window.partitionBy('k').orderBy(df.v.desc())

In [ ]:
df = df.withColumn('idx', f.monotonically_increasing_id())

df.select('k','v',
          f.dense_rank().over(Window.orderBy('k','v')).alias('rank_w1'),
          f.first('v',True).over(w).alias('first_w'),
          f.first('v',True).over(w1).alias('first_w1'),
          f.first('v',True).over(w2).alias('first_w2'),
          f.last('v',True).over(w).alias('last_w'),
          f.last('v',True).over(w1).alias('last_w1'),
          f.last('v',True).over(w2).alias('last_w2')).show()

In [ ]:
df = sp.sc.parallelize([(1,1,False),(2,1,False),(3,1,False),(4,1,True),(5,1,True),(6,1,True),(7,1,False),(8,1,False),(9,1,True),(10,1,True),
                        (11,2,False),(12,2,True),(13,2,True),(14,2,True),(15,2,False),(16,2,False),(17,2,False),(18,2,True),(19,2,True)]).toDF(['id','cid','flag'])

In [ ]:
w=Window.partitionBy('cid').orderBy('row_num')#.rowsBetween(Window.unboundedPreceding, Window.currentRow)

res = df.withColumn('new_col', f.when(df.flag == False,f.col('id')).otherwise(None))\
       .withColumn('idx', f.monotonically_increasing_id())\
       .withColumn('row_num',f.row_number().over(Window.orderBy('idx')))\
       .withColumn('last', f.last('new_col',True).over(Window.orderBy('cid','row_num')))\
       .withColumn('agg', f.coalesce(f.col('new_col'),f.last('new_col',True).over(Window.orderBy('cid','row_num'))))\
       .withColumn('agg_2', f.when(((f.lead(f.col('flag'),count=1).over(w)==False)&\
                                    (f.lead(f.col('flag'),count=2).over(w)==False))&(f.col('flag')!=True),
                                             f.lead(f.col('agg'),count=2).over(w))\
                             .when(((f.lead(f.col('flag'),count=1).over(w)==False)&\
                                    (f.lead(f.col('flag'),count=2).over(w)!=False))&(f.col('flag')!=True),
                                             f.lead(f.col('agg'),count=1).over(w))\
                             .otherwise(f.col('agg')))

In [ ]:
res.orderBy(f.col('id').asc(),f.col('cid').asc()).show(20)

In [ ]:
# w=Window.partitionBy('cid').orderBy(f.col('new_col_2').desc())

In [ ]:
res=res.withColumn('new_col_2', f.when(((f.col('cclead') == True)&\
                                        (f.lead(f.col('cclead')).over(Window.orderBy('row_num'))==False)),f.col('agg')).otherwise(None))
     #.withColumn('test',f.last('new_col_2',True).over(w))
     #.withColumn('agg_new', f.coalesce(f.col('new_col_2'),f.first('new_col_2',True).over(Window.orderBy('cid','row_num',f.col('new_col_2').desc())))) 

In [ ]:
# df = df.withColumn('add_new', f.when((df.flag == False)&(f.lead('flag').over(Window.orderBy('row_num')) == False),
#                                      f.lead('agg').over(Window.orderBy('row_num'))).otherwise(f.col('agg')))

### Save pyspark DF

In [ ]:
path_to_hdfs = "/user/aleksukhin-am_ca-sbrf-ru/sberbank_action_filter"

gadf_trimmed.coalesce(5).write \
            .format('com.databricks.spark.csv') \
            .mode('overwrite') \
            .option('sep', ';') \
            .option('header', True) \
            .option("quote", '\u0000') \
            .save(path_to_hdfs)